In [1]:
# APE
# c1 lower than lower bb
# c2 ema_200 > ema_500
# c3 ema 500 > ema_1000
# c4 Volume > previous Volume
# c5 Green Candle
# c6 previous Red Candle
# c7 lower than ema_500

In [2]:
import requests
import json
import pandas as pd
import datetime as dt
import time
import ta
import os

In [3]:
# 調整 notebook 寬物
# from IPython.display import display, HTML
# display(HTML("<style>.container { width:100% ! important; }<style>"))

# 調整 output 不折疊
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [4]:
import winsound
frequency = 250  # Set Frequency To 2500 Hertz
duration = 300  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [5]:
from binance.client import Client
from dotenv import load_dotenv

# Get the path to the current directory
current_directory = os.getcwd()

# Specify the path to the .env file relative to the current directory
dotenv_path = os.path.join(current_directory, '.env')

# Load the environment variables from the .env file
load_dotenv(dotenv_path)

api_key = os.getenv('API_KEY')
api_secret = os.getenv('SECRET_KEY')

client = Client(api_key, api_secret, testnet = False)

spot = client.get_account()
s_bal = pd.DataFrame(spot['balances'])
print(s_bal)

BinanceAPIException: APIError(code=-2015): Invalid API-key, IP, or permissions for action.

In [ ]:
timezone = -3
symbol = 'apeusdt'
interval = '15m'

# step between timestamps in milliseconds, 60000 = 1min 
step = 1000
start_time = round(time.time() * 1000 - (86400 * 1000 * 12 ))
end_time = round(time.time() * 1000)

def define_time():
    
    global start_time, end_time
    
    # start epoch till now, use prior 5 days for this strategy
    start_time = round(time.time() * 1000 - (86400 * 1000 * 12))
    end_time = round(time.time() * 1000)


In [ ]:
# 先抓歷史資料好讓技術指標能成型
def get_historical(symbol, interval, start_time, end_time, step):
    
    data = []    
    while start_time < end_time:
        limit = min(step, end_time - start_time + 1)  # Adjust the limit for the last batch
        while True:
            try:
                response = client.get_klines(symbol=symbol.upper(), interval=interval, limit=limit, startTime=start_time)
                break
            except:
                time.sleep(5)
        if len(response) == 0:
            break  # No more data available, exit the loop
        data.extend(response)
        start_time = response[-1][0] + 1
    
    # Convert the data to a DataFrame
    columns = [
        "Open_Time", "Open", "High", "Low", "Close", "Volume", "Close_Time",
        "Quote asset volume", "Number of trades", "Taker buy base asset volume",
        "Taker buy quote asset volume", "Ignore"
    ]
    raw_df = pd.DataFrame(data, columns=columns)   
    
    raw_df = raw_df[['Open_Time', 'Open', 'Close', "High", "Low", 'Volume']]
    convert_dict = {'Open_Time': float, 'Open': float, 'Close': float, "High": float, "Low": float, 'Volume': float}
    raw_df = raw_df.astype(convert_dict)

    raw_df['Open_Time'] = pd.to_datetime(raw_df['Open_Time'], unit = 'ms')
    raw_df['Open_Time'] = raw_df['Open_Time'] + pd.Timedelta(hours=timezone)
    raw_df['Open_Time'] = raw_df['Open_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    raw_df = raw_df.reset_index(drop=True)
    
    return raw_df

In [ ]:
raw_df = get_historical(symbol, interval, start_time, end_time, step)
print(raw_df)

In [ ]:
# def check_cross(df, kd_dir):
#     up = df['slow_k'] > df['slow_d']
#     down = df['slow_k'] < df['slow_d']
#     if kd_dir == 'Up':
#         return up.diff() & up
#     if kd_dir == 'Any':
#         return up.diff()
#     if kd_dir == 'Down':
#         return down.diff() & down


def indicators(df):

# ema
    for i in (200, 500, 1000):
        df['ema_'+str(i)] = ta.trend.ema_indicator(df.Close, window=i)

# atr
    df['atr'] = ta.volatility.average_true_range(df.High, df.Low, df.Close)

# bband
    bb_int = 30
    bb_dev = 2
    bb = ta.volatility.BollingerBands(df['Close'], window=bb_int, window_dev=bb_dev)
    df['bb_u'] = bb.bollinger_hband()
    df['bb_m'] = bb.bollinger_mavg()
    df['bb_l'] = bb.bollinger_lband()

# kd
#     df['slow_k']= ta.momentum.stoch(df['High'], df['Low'], df['Close'], 17, 5)
#     df['slow_d'] = ta.momentum.stoch_signal(df['High'], df['Low'], df['Close'], 17, 5)
    
# kd cross
#     df['kd_cross'] = check_cross(df, kd_dir)


In [ ]:
def conditions(df):
    df['c1'] = df['Low'] <= df['bb_l']
    df['c2'] = df['ema_200'] >= df['ema_500']
    df['c3'] = df['ema_500'] >= df['ema_1000']
    df['c4'] = df['Volume'] >= df['Volume'].shift(1)
    df['c5'] = df['Close'] >= df['Open']
    df['c6'] = df['Close'].shift(1) <= df['Open'].shift(1)
    df['c7'] = df['Low'] <= df['ema_500']
    
    # signal
    df['signal'] = df['c1'] & df['c2'] & df['c3'] & df['c4'] & df['c5'] & df['c6'] & df['c7']
            
    # 下一根進場
    df['entry'] = False

In [ ]:
def cancel_orders():
    
    open_orders = client.futures_get_open_orders()
    
    # Cancel all open orders
    if open_orders:
        # Cancel all open orders
        for order in open_orders:
            if (order['symbol'] == symbol.upper()) and (order['positionSide'] == 'LONG'):  
                cancel_response = client.futures_cancel_order(symbol=symbol.upper(), orderId=order['orderId'])
                print(f"Canceled order: {order['symbol']} - {order['orderId']}")
    else:
        # Continue with the rest of the code
        print("No open LONG orders found.")
        time.sleep(1)  # Sleep for 1 second to avoid API rate limit

In [ ]:
def enter_position(df):
        
    #-----Calculate entry price-----#
    close_val = df['Close']
    atr_val = df['atr']
    df.loc[df.index[-1], 'entry_p'] = close_val.loc[close_val.index[-2]]
    df.loc[df.index[-1], 'stop_loss'] = close_val.loc[close_val.index[-2]] - sl_atr * atr_val.loc[atr_val.index[-2]]
    df.loc[df.index[-1], 'take_profit'] = close_val.loc[close_val.index[-2]] + tp_atr * atr_val.loc[atr_val.index[-2]]

    #-----position attributes-----#
    quantity = 300
    entry_p = round(df.loc[df.index[-1], 'entry_p'],3)
    stop_loss_p = round(df.loc[df.index[-1], 'stop_loss'], 3)
    take_profit_p = round(df.loc[df.index[-1], 'take_profit'], 3)
    
    cancel_orders()

    try:
        order = client.futures_create_order(
            symbol=symbol.upper(),
            side='BUY',
            type='LIMIT',
            timeInForce='GTC',
            quantity=quantity,
            price=entry_p,
            positionSide='LONG',
        )
        print('Order created successfully.')       
        
        take_profit_order = client.futures_create_order(
            symbol=symbol.upper(),
            side='SELL',
            type='TAKE_PROFIT_MARKET',
            stopPrice=take_profit_p,
            stopLimitTimeInForce='GTC',
            closePosition = 'true',
            positionSide = 'LONG'
        )
        winsound.Beep(frequency, duration)
        print('Take profit created successfully')
            
    except Exception as e:
        print(f'Error creating order: {e}')
        
    print(str(symbol.upper()) + ' entered at ' + str(entry_p) + ', stop loss at ' + str(stop_loss_p))

    return df

In [ ]:
def check_sl(df, current_k):
    
    # 檢查前一根 Close 是否低於 SL，若低於，即刻停損
    
    last_signal_row = df.loc[df['signal'].eq(True) & df['Open_Time'].ne(current_k)].tail(1)
    if not last_signal_row.empty:
        current_sl = round(df.loc[last_signal_row.index[0], 'Close'] - df.loc[last_signal_row.index[0], 'atr'] * sl_atr, 3)
        current_tp = round(df.loc[last_signal_row.index[0], 'Close'] + df.loc[last_signal_row.index[0], 'atr'] * tp_atr, 3)

    print(f'Current SL at {current_sl}')
    print(f'Current TP at {current_tp}')
    
    # 確定停損
    if df.iloc[df.index[-2]]['Close'] <= current_sl:
        
        print('Last candle closed below SL, try create SL Order')
        
        cancel_orders()

        # 如果目前價格高於停損
        ticker = client.futures_symbol_ticker(symbol=symbol.upper())

        mark_price = round(float(ticker['price']), 3)
        
        stop_price = current_sl
        if current_sl > mark_price:
            stop_price = mark_price
        
        try:

            stop_loss_order = client.futures_create_order(
                symbol=symbol.upper(),
                side='SELL',
                type='STOP_MARKET',
                stopPrice=stop_price,
                stopLimitTimeInForce='GTC',
                closePosition = 'true',
                positionSide = 'LONG'
            )
        
            print(f'Stop loss created successfully at: {current_sl}')
            winsound.Beep(frequency, duration)
        
        except Exception as e:
            
            print(f'Error creating SL order: {e}')
          

In [ ]:
import IPython

def restart_and_run_all():
    # Restart the kernel
    IPython.Application.instance().kernel.do_shutdown(True)
    
    # Re-run all code cells
    IPython.get_ipython().run_line_magic('run', '-i 000_021_LONG_200_500_1000_15m.ipynb')


In [ ]:
from IPython.display import clear_output

line_count = 0
max_lines = 1
current_k = 0
restart_countdown = 2 * 60 * 60 # restart in 2 hours

def console_log(df):
    global line_count, max_lines, restart_countdown, loop_end_time
    try:
        df = df.reset_index(drop=True)
        df = df.round(3)
        print('----------------------------------------------------------------------------')
        print(f"{str(df.loc[df.index[-1], 'Open_Time'])} at {str(df.loc[df.index[-1], 'Close'])}")
        print()
        print(df[['Open_Time', 'Close', 'Volume', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'ema_1000', 'signal', 'entry']].tail())
        print('----------------------------------------------------------------------------')

        positions = client.futures_account()['positions']
        for position in positions:
            if float(position['positionAmt']) != 0:
                position_df = pd.DataFrame({'Symbol':position['symbol'],
                                            'Side':position['positionSide'],
                                            'Entry_P':round(float(position['entryPrice']),3),
                                            'Amt':round(float(position['positionAmt']) * df.loc[df.index[-1], 'Close'],3),
                                            'PL':round(float(position['unrealizedProfit']),3),
                                            'X':round(float(position['leverage']),1),
                                           }, index=[0])      
                print(position_df)
                print()

        # countdown for restarting session        
#         restart_countdown -= 1
#         print(f'{restart_countdown} iterations until restart')

#         if restart_countdown <= 100:
#             restart_and_run_all()
            
#         # count each loop time
        loop_end_time = time.time()
        print("Time taken to execute for loop:", loop_end_time - loop_start_time, "seconds")            
            
    except Exception as e:
        print(f'Error UPDATING info: {e}')    

    line_count += 1
    if line_count >= max_lines:
        clear_output(wait=True)
        line_count = 0

In [ ]:
def check_time(df):
    global current_k
    
    try:
        # 如果時間標籤改變，代表上個 Close 發生，檢查是否進場
        if df['Open_Time'][len(df) - 1] != current_k:

            print('time changed')
            current_k = df['Open_Time'][len(df) - 1]

            if df.iloc[df.index[-2]]['signal'] == True:

                winsound.Beep(frequency, duration)
                df.loc[df.index[-1], 'entry'] = True
                enter_position(df)

                print(df.tail(2))

    except Exception as e:
        print(f'Error creating ENTRY order: {e}')

In [ ]:
loop_start_time = 0
loop_end_time = 0

def run():
    global current_k, restart_countdown, loop_start_time
    df = pd.DataFrame()
    while True:
        loop_start_time = time.time()        
        try:
            define_time()
            while True:
                try:
                    df = get_historical(symbol, interval, start_time, end_time, step)
                    break
                except:
                    time.sleep(5)
            
            indicators(df)
            conditions(df)
            check_time(df)

            # 檢查停損
            try:
                positions_info = client.futures_account()['positions']
                long_positions = [p for p in positions_info if p['positionSide'] == 'LONG' and
                                  float(p['positionAmt']) != 0 and
                                  p['symbol'] == symbol.upper()]

                if long_positions:

                    check_sl(df, current_k)

                else:
                    print('No LONG position. No SL')

            except Exception as e:
                print(f'Error checking SL: {e}')

            # 更新狀態
            console_log(df)
            
            time.sleep(4)
            
        except ConnectionError as e:
            print("Connection error occurred:", e)
            print("Retrying in 5 seconds...")
            time.sleep(5)

In [ ]:
sl_atr = 4
tp_atr = 3
# kd_dir = 'Up'

In [ ]:
run()   

In [ ]:
# c1 Low lower than lower bb
# c2 ema_200 > ema_500
# c3 ema 500 > ema_1000
# c4 Volume > previous Volume
# c5 Green Candle
# c6 previous Red Candle
# c7 Low lower than ema_500